In [15]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms python-terrier

In [16]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="notebook")

In [17]:
from pathlib import Path

cache_dir = Path(f"./cache/")

In [18]:
from pathlib import Path
from examples.trec_util import TrecTrack

track = TrecTrack(
    28, "deep", "msmarco-passage/trec-dl-2019", cache_dir,
    Path(
        "/mnt/ceph/storage/data-in-progress/data-research/"
        "web-search/web-search-trec/trec-system-runs"
    )
)
retrieval_systems = [result % 10 for result in track.results]

Read results:   0%|          | 0/37 [00:00<?, ?run/s]

In [19]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    LNC1, TF_LNC, LB1, STMC1, STMC1_f, STMC2, STMC2_f, AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC,
    AndAxiom
)

# Define axiom for reranker and permutations.
axiom = AndAxiom([
    ~ArgUC(),
    ~QTArg(),
    ~QTPArg(),
    ~aSL(),
    ~LNC1(),
    ~TF_LNC(),
    ~LB1(),
    ~PROX1(),
    ~PROX2(),
    ~PROX3(),
    ~PROX4(),
    ~PROX5(),
    # ~REG(),  # Tie
    # ~ANTI_REG(),  # Tie
    ~AND(),
    ~LEN_AND(),
    ~M_AND(),
    ~LEN_M_AND(),
    ~DIV(),
    ~LEN_DIV(),
    ~RS_TF(),
    ~RS_TF_IDF(),
    ~RS_BM25(),
    ~RS_PL2(),
    ~RS_QL(),
    ~TFC1(),
    ~TFC3(),
    ~M_TDC(),
    ~LEN_M_TDC(),
    ~STMC1(),
    ~STMC1_f(),
    ~STMC2(),
    ~STMC2_f(),
])

In [20]:
from ir_axioms.backend.pyterrier.transformers import AxiomaticReranker

reranked_retrieval_systems = [
    ~(
            retrieval_system >> AxiomaticReranker(
        axiom=axiom,
        index=track.index,
        dataset=track.dataset_name,
        verbose=True
    )
    )
    for retrieval_system in retrieval_systems
]

In [21]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG

# noinspection PyTypeChecker
experiment = Experiment(
    [*reranked_retrieval_systems, *retrieval_systems],
    track.dataset.get_topics(),
    track.dataset.get_qrels(),
    [nDCG @ 10],
    verbose=True,
)
experiment

pt.Experiment:   0%|          | 0/74 [00:00<?, ?system/s]

Reranking axiomatically:   0%|          | 0/200 [00:00<?, ?query/s]

KeyboardInterrupt: 